In [3]:
import sys
sys.path.append(r"..")
from easy_helper import simplify_rllib_metrics
from ray.rllib.algorithms.ppo import PPOConfig

# 自定义环境
使用的数据分为 训练数据集/验证数据集

In [4]:
import gymnasium as gym
class CartPoleEnv(gym.Env):
    def __init__(self, *args, **kwargs):
        """
        CartPole 环境
        """
        self.env = gym.make('CartPole-v1', *args, **kwargs)
        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space

        # 数据类型 train/val
        print('init data type train')
        self.data_type = 'train'

    def val(self):
        print(f'set data type to val')
        self.data_type = 'val'
    
    def train(self):
        print(f'set data type to train')
        self.data_type = 'train'

    def reset(self, *args, **kwargs):
        return self.env.reset(*args, **kwargs)

    def step(self, *args, **kwargs):
        return self.env.step(*args, **kwargs)


In [5]:
# 注册环境
from ray.tune.registry import register_env
register_env('test_CartPole-v1', lambda config={}: CartPoleEnv())

In [27]:
from ray.rllib.algorithms.callbacks import DefaultCallbacks

def _get_env(env):
    while not isinstance(env, CartPoleEnv):
        env = env.unwrapped
    return env

class TestCallbacks(DefaultCallbacks):

    def on_evaluate_start(self, *args, **kwargs):
        print('on_evaluate_start')
        # 获取 eval_env_runner
        algo = kwargs['algorithm'] 
        if algo.eval_env_runner_group is None:
            eval_env_runner = algo.env_runner_group.local_env_runner
        else:
            eval_env_runner = algo.eval_env_runner
        # 切换环境到 val模式
        for env in eval_env_runner.env.unwrapped.envs:
            _env = _get_env(env)
            _env.val()

    def on_evaluate_end(self, *args, **kwargs):
        print('on_evaluate_end')
        # 获取 eval_env_runner
        algo = kwargs['algorithm'] 
        if algo.eval_env_runner_group is None:
            eval_env_runner = algo.env_runner_group.local_env_runner
            # 只有本地 eval_env_runner 需要切换回 train模式
            for env in eval_env_runner.env.unwrapped.envs:
                _env = _get_env(env)
                _env.train()

# 训练
再验证环节需要切换环境的数据类型 > 验证数据集

In [29]:
config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("test_CartPole-v1")
    .evaluation(
        evaluation_interval=1,
        evaluation_duration=1,
    )
    .callbacks(TestCallbacks)
)

algo = config.build()
simplify_rllib_metrics(algo.train())

2025-02-14 12:16:57,575	WARNING ppo.py:295 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
d:\programs\miniconda3\Lib\site-packages\ray\rllib\algorithms\algorithm.py:569: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
d:\programs\miniconda3\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ig

init data type train


2025-02-14 12:16:57,851	WARNING ppo.py:295 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html


init data type train


2025-02-14 12:16:58,236	WARNING rl_module.py:427 -- Could not create a Catalog object for your RLModule! If you are not using the new API stack yet, make sure to switch it off in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. Some algos already use the new stack by default. Ignore this message, if your RLModule does not use a Catalog to build its sub-components.
2025-02-14 12:16:58,285	WARNING util.py:61 -- Install gputil for GPU system monitoring.


on_evaluate_start
set data type to val
on_evaluate_end
--------- 训练迭代: 1 ---------
环境运行器:
  episode平均回报: 24.1700
  episode最大回报: 70.0000
  episode平均步数: 24.1700
  episode最大步数: 70.0000
  采样环境总步数: 4000.0000
  episodes计数: 177.0000

评估:
  episode平均回报: 19.0000
  episode最大回报: 19.0000
  episode平均步数: 19.0000
  episode最大步数: 19.0000
  无评估数据

学习者(默认策略):
  熵: 0.6719
  策略损失: -0.0457
  值函数损失: 6.6426
  总损失: 6.6014

本轮时间: 23.1346
每轮训练步数: 1
------------------------------


{'环境运行器': {'episode平均回报': 24.17,
  'episode最大回报': 70.0,
  'episode平均步数': 24.17,
  'episode最大步数': 70,
  '采样环境总步数': 4000,
  'episodes计数': 177},
 '评估': {'episode平均回报': 19.0,
  'episode最大回报': 19.0,
  'episode平均步数': 19.0,
  'episode最大步数': 19},
 '学习者': {'默认策略': {'熵': 0.6718525886535645,
   '策略损失': -0.045668601989746094,
   '值函数损失': 6.642553329467773,
   '总损失': 6.601372241973877}},
 '本轮时间': 23.1346492767334,
 '每轮训练步数': 1,
 '训练迭代次数': 1}